# Building your Deep Neural Network: Step by Step

첫 실습에서 구현했던 logistic regression을 확장해서 많은 계층을 갖는 (deep) neural network를 구현해봅니다. 


- 본 실습에서는, deep neural network를 구현하는데 필요한 기능들을 구현합니다.
- 본 실습에서 구현된 deep neural network는 다음 실습에서 고양이 사진을 구별하는데 사용됩니다. 


**본 실습을 완성하면 다음에 대해서 이해하게 됩니다.**
- ReLU같은 non-linear activation unit을 사용하여 모델을 개선합니다.
- 여러개의 hidden layers를 갖는 깊은 신경망을 구현합니다.
- neural network를 class로 구현하여 사용을 쉽게 합니다.

**Notation**:
- Superscript $[l]$ 는 $l$번째 계층을 나타내기 위해서 사용합니다. 
    - Example: $a^{[l]}$ 는 $l^{th}$ 계층의 activation입니다. $W^{[l]}$와 $b^{[l]}$는 $l$번째 계층의 파라미터(weights)입니다. 
- Superscript $(i)$는 $i^{th}$ example 입니다. 
    - Example: $x^{(i)}$는 $i^{th}$ training example입니다.
- Lowerscript $i$는 벡터에서 $i$번째 엔트리입니다.
    - Example: $a^{[l]}_i$는 $l^{th}$ 계층의 activations에서 $i$번째 엔트리입니다..



# 0 - 시작하기전에...
## 저 이제욱은 본 과제를 수행하면서 다른사람의 도움을 받거나 주지 않았습니다. 

## 1 - Packages

본 과제에는 다음의 패키지들이 필요합니다. 
- [numpy](www.numpy.org)는 Python에서 과학연산을 위한 기본 패키지입니다.
- [matplotlib](http://matplotlib.org)는 파이선에서 그래프를 그리기 위한 패키지입니다.
- dnn_utils 에는 `relu()`, `relu_backward()` 함수처럼 본과제에서 필요한 몇가지 유용한 함수를 제공합니다.
- testCases 는 구현이 잘되었는지 체크하는 함수들이 있습니다.
- np.random.seed(1)는 모든 실험이 동일한 결과를 얻을수있도록합니다. 


In [23]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4a import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2 - Outline of the Assignment

본 과제에서는 뉴럴 네트워크에서 필요한 여러가지 'helper'함수들을 구현합니다. 이러한 함수들은 다음 과제에서 2개 이상의 계층을 갖는 네트워크를 만들때 사용됩니다. 
각 함수를 구현할때 필요한 설명이 주어져 있으니, 먼저 잘 읽어보도록 하세요. 
다음과 같은 함수들을 구현할 것입니다.

1) $L$계층을 갖는 뉴럴네트워크의 파라미터를 초기화한다.
2) Forward propagation의 전체과정을 다음의 순서로 구현합니다. 
    - 먼저, $Z^{[\ell]}$을 출력하는 LINEAR 파트를 구현합니다. 
    - Non-linear ACTIVATIOPN 함수인 relu와 sigmoid의 구현은 dnn_utils_v2.py에 이미 구현되어있습니다. 
    - LINEAR한 부분과 ACTIVATION을 결합하여, [LINEAR->ACTIVATION]을 처리하는 함수를 구현합니다. 
    - [LINEAR->RELU]를 $L-1$번 반복하여 결합하고, [LINEAR->SIGMOID]를 마지막에 결합한 L_model_forward 함수를 구현합니다. 이 함수는 forward propatation의 전 과정을 수행합니다.

3) loss (또는 cost)를 구하는 함수를 구현합니다. 

4) Backward propagation을 다음 순서로 구현합니다. 
    - backward propagation의 LINEAR한 파트를 구현합니다.
    - Non-linear ACTIVATION 함수의 backward propagation은 relu_backward/sigmoid_backward에 이미 구현되어 있습니다. 
    - 앞의 두 과정을 결합하여, [LINEAR->ACTIVATION]에 대한 backward propagation을 수행하는 함수를 구현합니다. 
    - [LINEAR->RELU]를 $L-1$번 반복하여 결합하고, [LINEAR->SIGMOID]를 마지막에 결합한 L_model_backward 함수를 구현합니다. 이 함수는 backward propatation의 전 과정을 수행합니다.


<img src="images/final outline.png" style="width:800px;height:500px;">


**Note:** 모든 forward함수는 이에 대응하는 backward함수가 필요합니다. 따라서, forward함수에서는 backward함수에서 gradients를 계산할때 필요한 값들을 cache에 저장하여 사용합니다. 

## 3 - Initialization

$L$ 계층을 갖는 네트워크를 초기화 합니다. 네트워크의 모든 파라미터들($W^{[l]}$과 $b^{[l]}$)을 초기화 합니다. 

**Instructions**:
- 텐서를 랜덤값으로 초기화 하기 위해서는 `np.random.randn(shape)*0.01`을 사용합니다. 0으로 초기화하지 않는 이유에 대해서 생각해보세요.
- bias는 0으로 초기화 합니다. `np.zeros(shape)`를 사용합니다. 

### 3.2 - L-layer Neural Network


$L$계층을 갖는 네트워크에서 $\ell$번째 계층은 $n^{[\ell]}$개의 유닛(뉴런)을 갖습니다. 따라서, 입력이 $m=209$개의 examples를 갖는 경우 입력 $X$의 모양은 $(12288, 209)$이며, 각 계층의 파라미터와 activation의 모양은 다음과 같습니다. 

|               | Shape of **W**   |  Shape of **b** |  **Activation**  | **Shape of Activation**   |
|---------------|------------------|-----------------|------------------|---------------------------|
|**Layer 1**    | $(n^{[1]},12288)$| $(n^{[1]},1)$   |  $A^{[1]} = \sigma(W^{[1]}  X + b^{[1]})$ |  $(n^{[1]},209)$   |
|**Layer 2**    | $(n^{[2]},n^{[1]})$| $(n^{[2]},1)$   |  $A^{[2]} = \sigma(W^{[2]}  X + b^{[2]})$ |  $(n^{[2]},209)$   |
| ...           | ...              |                 |                  |                            |
|**Layer L-1**    | $(n^{[L-1]},n^{[L-2]})$| $(n^{[L-1]},1)$   |  $A^{[L-1]} = \sigma(W^{[L-1]}  X + b^{[L-1]})$ |  $(n^{[L-1]},209)$   |
|**Layer L**    | $(n^{[L]},n^{[L-1]})$| $(n^{[L]},1)$   |  $A^{[L]} = \sigma(W^{[L]}  X + b^{[L]})$ |  $(n^{[L]},209)$   |


파이선에서 $W X + b$를 계산할때 broadcating을 통해 모양을 맞추는것을 기억하세요. 
예를 들어서...

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

$WX + b$를 계산하면 아래와 같습니다. $b$가 column의 개수 만큼 broadcating되어서 각 column에 더해집니다.

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**Exercise**: L-layer Neural Network의 파라미터를 초기화합니다.

**Instructions**:
- 모델은 다음의 구조를 갖습니다. *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. 즉, ReLU를 activation으로 갖는 $L-1$개의 계층과 sigmoid를 activation으로 갖는 마지막 계층으로 구성됩니다. 
- W는 랜덤으로 초기화 합니다. `np.random.randn(shape) * 0.01`를 사용하세요.
- bias는 0으로 초기화합니다. `np.zeros(shape)`를 사용하세요. 
- 함수에 인자로 주어지는 `layer_dims`는 각 계층의 뉴런의 개수 $n^{[l]}$를 갖는 리스트입니다. 예를 들어 [2,4,1]는 2개의 입력, 첫번째 hidden layer에는 4개의 뉴런, 마지막 출력은 1개인 네트워크를 나타냅니다. 이에 따라서, $W^{[1]}$의 모양은 (4,2), $W^{[2]}$의 모양은 (1,4), $b^{[1]}$의 모양은 (4,1), $b^{[2]}$의 모양은 (1,1)입니다. $L$계층의 네트워크에서는 이를 일반화시켜서 생각합니다.
- 만약에 $L=1$인 1계층만 갖는 네트워크라면, 다음처럼 초기화 할것입니다. $L$이 1보다 큰 일반적인 경우에 대해서 구현하세요. 
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```


In [24]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)  # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
        ### END CODE HERE ###

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters


In [25]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> W1 </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>b1 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>W2 </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>b2 </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4 - Forward propagation module

### 4.1 - Linear Forward 
네트워크의 파라미터를 초기화 했으니, 이제는 forward propagation함수를 구현합니다. 
다음의 함수를 단계적으로 구현합니다. 
- LINEAR
- LINEAR -> ACTIVATION : 여기서 ACTIVATION은 ReLU 또는 Sigmoid입니다.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID :이것이 전체 모델입니다.

LINEAR 부분은 다음의 연산을 수행합니다. 

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

여기서 $A^{[0]} = X$입니다. $A^{[0]}$는 모든 examples를 가지고 있는 매트릭스(rank-2 텐서)입니다.

**Exercise**: forward propagation의 LINEAR파트를 구현합니다. 

**Reminder**:
$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$를 계산하기 위해서 `np.dot()`을 사용하세요. 텐서의 차원이 맞는지 확인하기 위해서는 `print(W.shape)`처럼 모양을 찍어서 디버깅합니다.

In [26]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W,A)+b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [27]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**Expected output**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4.2 - Linear-Activation Forward

이번 과제에서는 2가지의 activation 함수가 사용됩니다. 

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. 
`sigmoid()`함수는 이미 구현되어 있습니다. 이 함수는 2개의 값을 반환합니다. "`A`"와 "`cache`"입니다. `cache`에는 "`Z`" 값을 갖습니다. `sigmoid`함수는 다음과 같이 사용합니다. 
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: RELU(Z) = max(0, Z)$. `relu()`함수도 이미 구현되어 있습니다. 이 함수는 2개의 값을 반환합니다. "`A`"와 "`cache`"입니다. `cache`에는 "`Z`" 값을 갖습니다. `relu`함수는 다음과 같이 사용합니다. 
``` python
A, activation_cache = relu(Z)
```


LINEAR과 ACTIVATION을 두개를 합쳐서 하나의 함수로 사용하는 것이 편리합니다. 
LINEAR를 먼저 호출하고, 그 결과를 ACTIVATION에 적용합니다. 

**Exercise**: 이 함수는 *LINEAR->ACTIVATION*를 수행합니다. 이를 수식으로 표현하면 다음과 같습니다: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$. 여기서 "$g$"는 sigmoid()또는 relu() 입니다. 

In [28]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
            stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        A, activation_cache = sigmoid(np.dot(W,A_prev)+b)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        A, activation_cache = relu(np.dot(W,A_prev)+b)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [29]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Expected output**:
       
<table style="width:35%">
  <tr>
    <td> With sigmoid: A </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> With ReLU: A </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>


**Note**: 딥러닝에서는 "[LINEAR->ACTIVATION]"를 1개의 계층(layer)로 카운트합니다. 2개 계층이 아닙니다. 파라미터가 없는 연산은 계층으로 카운트하지 않는것이 일반적입니다.

### d) L-Layer Model 

$L$계층의 뉴럴 네트워크에서는 ReLU를 사용하는 `linear_activation_forward`를 $L-1$번 반복하고, 마지막에는 SIGMOID를 사용하는 `linear_activation_forward`를 호출하는 하나의 모델로 만들게 됩니다. 

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">

**Exercise**: 위의 모델을 함수로 구현합니다. 

**Instruction**: 아래 코드에서 `AL`은 $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$입니다. 즉, `AL`은 최종 예측값인 $\hat{Y}$입니다. 

**Tips**:
- 앞에서 구현한 함수들을 사용해서 구현합니다. 
- 루프에서는 [LINEAR->RELU]를 (L-1)번 호출합니다.
- forward에서는 cache를 계속해서 리스트에 추가해야합니다. `caches`라는 리스트에 `c`를 추가하기 위해서는 `caches.append(c)`라고 하면 됩니다.

In [30]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A,cache = linear_activation_forward(A_prev,parameters['W' + str(l)],parameters['b' + str(l)],'relu')
        caches.append(cache)
        ### END CODE HERE ###
    
    A_prev = A 
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL,cache = linear_activation_forward(A_prev,parameters['W' + str(L)],parameters['b' + str(L)],'sigmoid')
    caches.append(cache)
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [31]:
X, parameters = L_model_forward_test_case_2hidden()
# print(np.dot(parameters['W1'],X)+parameters['b1'])
# print(linear_activation_forward(X,parameters['W1'],parameters['b1'],relu))
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


<table style="width:50%">
  <tr>
    <td> AL </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td> Length of caches list </td>
    <td > 3 </td> 
  </tr>
</table>

이제 forward propagation함수가 모두 완성되었습니다. 입력 X가 주어지면 마지막 출력인 $A^{[L]}$는 `m`개의 입력에 대한 예측값을 갖고 있습니다. 
각 계층에서의 중간 계산값들은 모두 "`caches`"에 저장되어 backward propagation에 사용되게 됩니다. 

## 5 - Cost function

현재 모델이 얼마나 잘하고 있는지를 나타내는 cost함수를 계산합니다. 

**Exercise**: cost함수 $J$를 계산하기 위해서는 다음 식으로 주어진 cross-entropy 함수를 사용합니다: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} \{y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)\} \tag{7}$$ 
이 함수는 미리 구현을 해두었습니다.


In [32]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = -1./m * np.sum((Y * np.log(AL) + (1-Y) * np.log(1 - AL)), axis=1, keepdims=True)
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [33]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.2797765635793422


**Expected Output**:
<table>
    <tr>
    <td>cost</td>
    <td> 0.2797765635793422</td> 
    </tr>
</table>

## 6 - Backward propagation module

forward propagation과 마찬가지로, backward propagation을 위한 함수들을 순차적으로 구현합니다. 
backward propagation에서는 파라미터에 대한 gradient를 구합니다. 


<img src="images/backprop_kiank.png" style="width:650px;height:250px;">

backward propagation에서는 다음의 3단계로 진행됩니다. 
- LINEAR backward
- LINEAR -> ACTIVATION backward: 여기서 ACTIVATION은 ReLU 또는 sigmoid입니다.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (모델 전체)

### 6.1 - Linear backward

계층 $l$에서, LINEAR파트는 $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$에 해당합니다. 

$dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$입니다. $dZ^{[l]}$는 이미 계산되었고, backward propagation되어 전달된 상태입니다. 전달받은 $dZ^{[l]}$를 이용해서 $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$를 구합니다.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption>Figure 4</caption>

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


**Exercise**: 위의 3개 식을 이용하여 `linear_backward()`함수를 구현합니다.

In [34]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    # Here cache is "linear_cache" containing (A_prev, W, b) coming from the forward propagation in the current layer
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [35]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]


**Expected Output**:
    
```
dA_prev = 
 [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = 
 [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = 
 [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
```

### 6.2 - Linear-Activation backward

 위에서 구현한 `linear_backward` 함수를 이용해서 `linear_activation_backward`를 구현합니다. 

 $g(.)$가 activation함수라면, `dZ`는 다음과 같이 계산됩니다:
$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$.  

 `linear_activation_backward`구현에 필요한 activation에 대한 backward함수는 다음처럼 이미 구현되어 제공됩니다. 
 
- `sigmoid_backward`: SIGMOID 에 대한 backward함수는 다음처럼 사용합니다:

```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward`**: RELU에 대한 backward함수는 다음처럼 사용합니다:

```python
dZ = relu_backward(dA, activation_cache)
```


In [36]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = relu_backward(dA,activation_cache)
        ### END CODE HERE ###

    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = sigmoid_backward(dA,activation_cache)
        ### END CODE HERE ###
    
    ### START CODE HERE ### (≈ 1 line of code)
    dA_prev, dW, db= linear_backward(dZ,linear_cache)
    ### END CODE HERE ###
    
    return dA_prev, dW, db

In [37]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Expected output with sigmoid:**

<table style="width:100%">
  <tr>
    <td>
        dA_prev
     </td> 
     <td>
         [[ 0.11017994  0.01105339]
         [ 0.09466817  0.00949723]
         [-0.05743092 -0.00576154]]
      </td> 
  </tr> 
    <tr>
        <td>
            dW
        </td> 
        <td>
            [[ 0.10266786  0.09778551 -0.01968084]]
        </td> 
  </tr> 
    <tr>
        <td>
        db
       </td> 
       <td >
           [[-0.05729622]]
        </td> 
  </tr> 
</table>

**Expected output with relu:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 

  </tr> 
  <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
  <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>



### 6.3 - L-Model Backward 

이제 전체 네트워크에 해당하는 backward 함수를 구현합니다. 


`L_model_forward`함수를 구현하면서 각 계층은 cache에 (X,W,b, and z)를 저장해 둔것을 기억해야합니다. 
backward propagation에서는 이 cache에 저장된 값들을 사용합니다. $L$번 계층에서 시작하여 hidden layers들을 역방향으로 처리합니다. 


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  Figure 5 : Backward pass  </center></caption>

backward propagation을 위해서는 가장 먼저 `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$를 계산해야합니다. 
이것은 다음처럼 하면 됩니다. 이미 구현해 두었으므로 확인만 하세요. 
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```
`dAL`을 구한후에는 
LINEAR->SIGMOID backward함수에 `dAL`을 입력으로 주게 됩니다. 

그리고, for 루프에서 LINEAR->RELU 계층들에 대한 backward propagation을 진행합니다. 각 계층에서는 dA, dW, db를 `grads` 딕셔너리에 저장합니다. 
딕셔너리에 저장하기 위해서는 다음처럼 합니다:

$grads["dW" + str(l)] = dW^{[l]}$

예를 들어, $l=3$일때는  `grads["dW3"]`에 $dW^{[3]}$을 저장합니다. 


In [38]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = - (np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)]= linear_activation_backward(dAL,caches[L-1],'sigmoid')
    ### END CODE HERE ###
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        grads["dA" + str(l)], grads["dW" + str(l+1)], grads["db" + str(l+1)]= linear_activation_backward(grads["dA" + str(l+1)],caches[l],'relu')
        ### END CODE HERE ###

    return grads

In [39]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


**Expected Output**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
  <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td> 

  </tr> 
</table>



### 6.4 - Update Parameters

모델의 모든 파라미터들에 대한 gradients를 구했으니, 이제 파라미터들의 값을 gradient descent를 이용해 업데이트합니다:

- $ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$
- $ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$

여기서, $\alpha$는 learning rate입니다. 업데이트된 파라미터는 `parameters` 딕셔너리에 저장합니다. 

**Instructions**:
모든 계층 $l (l=1,2,...,L$)의 파라미터 $W^{[l]}$ and $b^{[l]}$에 대해서 gradient descent를 합니다.


In [40]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                parameters["W" + str(l)] = ... 
                parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 2 lines of code)
    for l in range(1, L+1):
        parameters["W" + str(l)] =  parameters["W" + str(l)] - learning_rate * grads["dW" + str(l)]
        parameters["b" + str(l)] =  parameters["b" + str(l)] - learning_rate * grads["db" + str(l)]
    ### END CODE HERE ###
    return parameters

In [41]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**Expected Output**:

<table style="width:100%"> 
    <tr>
    <td >W1</td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>



## 7 - Conclusion


이제 딥 뉴럴네트워크를 위한 모든 함수를 구현하였습니다. 
지금 구현한 딥 뉴럴네트워크를 다음 과제에서 cat vs. non-cat과 같은 분류기를 훈련시키는데 적용할 것입니다.